In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [59]:
vals = []
with open('./input.txt', 'r') as file:
    for line in file:
        vals.append(line.strip())
        
vals[:5]

['3,6,12', '16,15,16', '14,4,14', '6,2,7', '4,8,16']

In [52]:
test ='''2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5'''

vals = []
for line in test.split('\n'):
    vals.append(line.strip())

vals[:5]

['2,2,2', '1,2,2', '3,2,2', '2,1,2', '2,3,2']

In [60]:
min_x, min_y, min_z = 100,100,100
max_x, max_y, max_z = -100,-100,-100
landscape = np.zeros((22,22,22), dtype=int)
for v in vals:
    x,y,z = v.split(',')
    x,y,z = int(x),int(y),int(z)


    landscape[x,y,z] = 1
    
    if x < min_x:
        min_x = x
    elif x > max_x:
        max_x = x
        
    if y < min_y:
        min_y = y
    elif y > max_y:
        max_y = y

    if z < min_z:
        min_z = z
    elif z > max_z:
        max_z = z 
        

(min_x, max_x), (min_y,max_y), (min_z,max_z)

((0, 21), (0, 21), (0, 21))

In [61]:
tots = 0
for cube in np.argwhere(landscape==1):
    assert min(cube) == np.abs(min(cube)), f"We have a -ve number {cube}"
    assert max(cube) <= 22, f"Landscape too small {cube}"
    touching = -3 + np.sum(landscape[np.max([min_x,cube[0]-1]):cube[0]+2,cube[1], cube[2]]) + \
        np.sum(landscape[cube[0], np.max([min_y,cube[1]-1]):cube[1]+2, cube[2]]) + \
        np.sum(landscape[cube[0], cube[1], np.max([min_z,cube[2]-1]):cube[2]+2])
    touching = (touching * -1) + 6
    tots += touching
tots

4628

In [100]:
def expand(check):
    '''
    state == 0 means air (untested)
    state == 1 means lava
    state == 2 means released to open air (and should be counted)
    state == 3 means enclosed (and should be excluded)
    '''

    smoke = [[check[0],check[1],check[2]],]
    
    state = 0
    explored = np.array([])
    
    wtf = 0
    wtf_smoke = []

    while len(smoke)>0:
        wtf += 1
        assert wtf < 100, "WTF"
        new_smoke = []
        for s in smoke:
            x,y,z = s
            do_it = False
            if len(explored)==0:
                explored = np.array([[x,y,z],])
                do_it = True
            else:
                if not (s==explored).all(1).any():
                    explored = np.vstack([explored, [x,y,z]])
                    do_it = True
                    
            if not do_it:
                continue


            if fakescape[np.max([min_x,x-1]),y,z] == 0:
                if len(new_smoke)==0:
                    new_smoke = np.array([[np.max([min_x,x-1]),y,z],])
                elif not ([np.max([min_x,x-1]),y,z] == explored).all(1).any():          
                    new_smoke = np.vstack([new_smoke, [np.max([min_x,x-1]),y,z]])

                
            if fakescape[np.min([max_x,x+1]),y,z] == 0:
                if len(new_smoke)==0:
                    new_smoke = np.array([[np.min([max_x,x+1]),y,z],])
                elif not ([np.min([max_x,x+1]),y,z] == explored).all(1).any():          
                    new_smoke = np.vstack([new_smoke, [np.min([max_x,x+1]),y,z]])


            if fakescape[x,np.max([min_y,y-1]),z] == 0:
                if len(new_smoke)==0:
                    new_smoke = np.array([[x,np.max([min_y,y-1]),z],])
                elif not ([x,np.max([min_y,y-1]),z] == explored).all(1).any():          
                    new_smoke = np.vstack([new_smoke, [x,np.max([min_y,y-1]),z]])

                
            if fakescape[x,np.min([max_y,y+1]),z] == 0:
                if len(new_smoke)==0:
                    new_smoke = np.array([[x,np.min([max_y,y+1]),z],])
                elif not ([x,np.min([max_y,y+1]),z] == explored).all(1).any():        
                    new_smoke = np.vstack([new_smoke, [x,np.min([max_y,y+1]),z]]) 

            if fakescape[x,y,np.max([min_z,z-1])] == 0:
                if len(new_smoke)==0:
                    new_smoke = np.array([[x,y,np.max([min_z,z-1])],])
                elif not ([x,y,np.max([min_z,z-1])] == explored).all(1).any():             
                    new_smoke = np.vstack([new_smoke, [x,y,np.max([min_z,z-1])]])

                    
            if fakescape[x,y,np.min([max_z,z+1])] == 0:
                if len(new_smoke)==0:
                    new_smoke = np.array([[x,y,np.min([max_z,z+1])],])
                elif not ([x,y,np.min([max_z,z+1])] == explored).all(1).any():          
                    new_smoke = np.vstack([new_smoke, [x,y,np.min([max_z,z+1])]])  
        
        if len(new_smoke)>0:
            if len(new_smoke[(((new_smoke <= min_x).any()) or ((new_smoke >= max_x).any()))]) > 0:
                state = 2

            smoke = new_smoke[((new_smoke >= min_x).all(1)) & ((new_smoke <= max_x).all(1))]

            
            wtf_smoke = new_smoke[((new_smoke <= min_x).any()) or ((new_smoke >= max_x).any())]
        else:
            smoke = []
            
    if state == 0 and len(wtf_smoke)==0:     
        state = 3
        
    else:
        state = 2
    return explored,state, wtf_smoke

In [101]:
fakescape = landscape.copy()
counter = 0

empties = np.argwhere(fakescape==0)
worth_checking = empties[(((empties > np.array([[min_x,min_y,min_z],])).all(1)) & \
                          ((empties < np.array([[max_x,max_y,max_z],])).all(1)))]     


while len(worth_checking) !=0:    
    checked, state,wtf = expand(worth_checking[0])

    for c in checked:
        fakescape[c[0],c[1],c[2]] = state        


    worth_checking = []
    empties = np.argwhere(fakescape==0)
        
    worth_checking = empties[(((empties > np.array([[min_x,min_y,min_z],])).all(1)) & \
                              ((empties < np.array([[max_x,max_y,max_z],])).all(1)))]     


In [112]:
newscape = np.zeros_like(fakescape)
for coords in np.argwhere(fakescape==3):
    newscape[coords[0],coords[1],coords[2]] = 1
    
to_remove = 0
for cube in np.argwhere(newscape==1):
    assert min(cube) == np.abs(min(cube)), f"We have a -ve number {cube}"
    assert max(cube) <= 29, f"Landscape too small {cube}"
    assert min(cube) > 0
    touching = -3 + np.sum(newscape[cube[0]-1:cube[0]+2,cube[1], cube[2]]) + \
        np.sum(newscape[cube[0], cube[1]-1:cube[1]+2, cube[2]]) + \
        np.sum(newscape[cube[0], cube[1], cube[2]-1:cube[2]+2])
    touching = (touching * -1) + 6
    to_remove += touching
tots - to_remove

2582